<h1>Perform dispatch model/Grimsel runs<span class="tocSkip"></span></h1>

Companion notebook #3 of 4 of the paper 
>M.C.Soini *et al.*, **On the market displacement of incumbent grid-connected electricity storage by more efficient storage**.

This notebook performs model runs discussed in section 3.3 of the paper.
Static HTML files are included with the paper's supplementary material.The analysis of the model results is performed in a separate [Jupyter Notebook](https://github.com/mcsoini/storage_displacement_supplementary/blob/master/04_dispatch_model_analysis.ipynb).

**Instructions**
* Grimsel requires a working CPLEX installation in a default location
* Install dependencies as defined in the README

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span><ul class="toc-item"><li><span><a href="#Grimsel-imports" data-toc-modified-id="Grimsel-imports-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Grimsel imports</a></span></li></ul></li><li><span><a href="#Define-parameters" data-toc-modified-id="Define-parameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define parameters</a></span></li><li><span><a href="#Read-input-data" data-toc-modified-id="Read-input-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Read input data</a></span></li><li><span><a href="#Model-input-table-adjustments" data-toc-modified-id="Model-input-table-adjustments-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Model input table adjustments</a></span><ul class="toc-item"><li><span><a href="#Raise-demand-by-grid-losses-and-set-grid-losses-to-0" data-toc-modified-id="Raise-demand-by-grid-losses-and-set-grid-losses-to-0-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Raise demand by grid losses and set grid losses to 0</a></span></li><li><span><a href="#Adjust-PHS-properties" data-toc-modified-id="Adjust-PHS-properties-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Adjust PHS properties</a></span></li></ul></li><li><span><a href="#Build-full-model" data-toc-modified-id="Build-full-model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Build full model</a></span></li><li><span><a href="#Model-component-adjustments" data-toc-modified-id="Model-component-adjustments-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Model component adjustments</a></span></li><li><span><a href="#Define-the-model-modifications" data-toc-modified-id="Define-the-model-modifications-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Define the model modifications</a></span></li><li><span><a href="#Run-the-model" data-toc-modified-id="Run-the-model-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Run the model</a></span></li></ul></div>

# Imports

In [ ]:
import sys, os, socket
import numpy as np
import pandas as pd
from importlib import reload
import itertools
import pyomo.environ as po

## Grimsel imports

In [ ]:
import grimsel.core.model_base as model_base
import grimsel.core.model_loop as model_loop
from grimsel.core.model_base import ModelBase as MB
from grimsel.core.io import IO as IO
import grimsel.auxiliary.maps as maps
from grimsel.auxiliary.aux_m_func import set_to_list
from grimsel.core.parameters import ParameterAdder
from grimsel.core.parameters import Par

from grimsel import logger

logger.setLevel('DEBUG')

path_csv = os.path.abspath('MODEL_INPUT_CSV_FILES/')

# Define parameters

In [ ]:
sc_out = 'out_value_test_scenonly'
sc_out = 'out_value_test_all6hours'
sc_out = 'compare_repro'

In [ ]:
# Model Parameters
mkwargs = {
           'symbolic_solver_labels': False,  # pyomo parameter, set to True for debugging
            # select constraint groups; the default objective function is skipped in order to add a custom one (see below)
           'constraint_groups': MB.get_constraint_groups(excl=['ror', 'variables', 'objective']),
           'nthreads': 3,  # CPLEX multiprocessing
           'keepfiles': False,  # if True: keep temporary files
           }

# Additional I/O parameters 
iokwargs = {'output_target': 'fastparquet',  
            'cl_out': sc_out,  # output collection name, here: path for parquet files
            'autocomplete_curtailment': False,  # if False: don't add curtailment in all nodes
            'data_path': path_csv,  # input data csv files
            'dev_mode': False  # if False: prompt warning if overwriting existing results
           }

# Specify names and numbers of model variations
nsteps_default = [('swfy', 8, np.arange),    # 8 future years [2015, 2020, ..., 2050]
                  ('swfz', 3, np.arange),    # Freeze cap (1), vc (2), or default (0)
                  ('swst', 31, np.arange),   # vary new storage capacity in 31 steps
                  ('swec', 2, np.arange),    # energy cost of storage (\gamma_e)
                 ]
# Additional model loop parameters
mlkwargs = {'nsteps': nsteps_default,
            'full_setup': False
            }

ml = model_loop.ModelLoop(**mlkwargs, mkwargs=mkwargs, iokwargs=iokwargs)
self = ml.m

# Define values for the identification of model run variations.
# The keys refer to the index column values in the ml.df_def_run table
# The values are used below in the methods of the ModelLoopModifier class
dict_fz = {0: 'default', 1: 'frz_cap', 2: 'frz_vc'}
dict_fy = {0: 2015, 1: 2020, 2: 2030, 3: 2040, 4: 2050,
           5:  2025, 6:  2035, 7:  2045}
# defines new storage share of incumbent storage energy capacity
dict_st = dict(enumerate(map(lambda x: int(x*10) / 10, np.linspace(0, 3, 31))))
dict_ec = {0: 'early_phs', 1: 'early_bat'}  # relative energy cost



# Read input data 
* Initialize model up to the point where all input data is read and adjusted (e.g. temporal averaging)

In [ ]:
ml.build_model(to_runlevel='input_data')

# Model input table adjustments

## Raise demand by grid losses and set grid losses to 0
* Facilitates analysis 
* Affects profile (default year) and total demand future years
* Permanent adjustments are made to the input tables (instead of the Pyomo components) in order to preserve them even when calling `reset_all_parameters`

In [ ]:
df_gl = ml.m.df_node_encar.set_index('dmnd_pf_id').grid_losses
ml.m.df_profdmnd = ml.m.df_profdmnd.join(df_gl, on='dmnd_pf_id')
ml.m.df_profdmnd['value'] = (ml.m.df_profdmnd['value'] * (1 + ml.m.df_profdmnd.grid_losses))
ml.m.df_profdmnd = ml.m.df_profdmnd.drop('grid_losses', axis=1)

cols_dmnd_sum = [c for c in ml.m.df_node_encar.columns if 'dmnd_sum' in c]
ml.m.df_node_encar[cols_dmnd_sum] = (
        ml.m.df_node_encar[cols_dmnd_sum]
            .multiply(1 + ml.m.df_node_encar.grid_losses, axis=0))

ml.m.df_node_encar['grid_losses'] = 0

## Adjust PHS properties
* Harmonize PHS properties in all countries

In [ ]:
# Set PHS efficiency to 75% consistently in all countries
list_HYD_STO = ml.m.mps.pp2pp('HYD_STO')
ml.m.df_plant_encar.loc[ml.m.df_plant_encar.pp_id.isin(list_HYD_STO), 'st_lss_rt'] = 0.25
# Set PHS discharge duration to 6 hours
ml.m.df_plant_encar.loc[ml.m.df_plant_encar.pp_id.isin(list_HYD_STO), 'discharge_duration'] = 6

# Build full model
Proceed with the initialization of the model after the modification of the input dataframes. `build_model` continues where it last stopped.

In [ ]:
ml.build_model()

# Model component adjustments
* Define a new parameter for the stored energy cost and include it in the objective function.

In [ ]:
ml.m.df_plant_encar['ec'] = np.where(
        ml.m.df_plant_encar.pp_id.isin(list(ml.m.st)),
        ml.m.mps.id_to_name(
                ml.m.df_plant_encar).pt.replace(
                        {'NEW_STO': 1e-4, 'HYD_STO': 1e-3}), 0)

parec = Par('ec', (ml.m.st, ml.m.ca), 'df_plant_encar', 'ec', ['pp_id'], ml.m.st, default=0)
parameter = ParameterAdder(ml.m, parec)
ml.m.dict_par[parec.parameter_name] = parameter
parameter.init_update()

self = ml.m

def objective_rule_quad(self):

    nnn, nn = [None] * 3, [None] * 2
    return (# FUEL COST CONSTANT
            sum(self.vc_fl_pp_yr[pp, ca, fl]
                * self.nd_weight[self.mps.dict_plant_2_node_id[pp]]
                for (pp, ca, fl)
                in set_to_list(self.pp_cafl - self.lin_cafl, nnn))
            # FUEL COST LINEAR
          + self.get_vc_fl()
            # EMISSION COST LINEAR
          + self.get_vc_co()
          + sum(self.vc_co2_pp_yr[pp, ca]
                * self.nd_weight[self.mps.dict_plant_2_node_id[pp]]
                for (pp, ca) in set_to_list(self.pp_ca - self.lin_ca, nn))
          + sum(self.vc_om_pp_yr[pp, ca]
                * self.nd_weight[self.mps.dict_plant_2_node_id[pp]]
                for (pp, ca) in set_to_list(self.ppall_ca, nn))
          + sum(self.vc_ramp_yr[pp, ca]
                * self.nd_weight[self.mps.dict_plant_2_node_id[pp]]
                for (pp, ca) in set_to_list(self.rp_ca, nn))
          + sum(self.fc_om_pp_yr[pp, ca]
                * self.nd_weight[self.mps.dict_plant_2_node_id[pp]]
                for (pp, ca) in set_to_list(self.ppall_ca, nn))
          + sum(self.fc_cp_pp_yr[pp, ca]
                * self.nd_weight[self.mps.dict_plant_2_node_id[pp]]
                for (pp, ca) in set_to_list(self.add_ca, nn))

          # ADDITIONAL COST ON STORAGE ENERGY CONTENT
          + sum(self.erg_st[sy, pp, ca] * self.ec[pp, ca]
                for sy, pp, ca in set_to_list(self.sy_st_ca, [None, None, None]))
          )

self.cadd('objective_quad', rule=objective_rule_quad,
          sense=po.minimize, objclass=po.Objective)

# Define the model modifications

In [ ]:
import wrapt

class ModelLoopModifier():
    '''
    The purpose of this class is to modify the parameters of the BaseModel
    objects in dependence on the current run_id.

    This might include the modification of parameters like CO2 prices,
    capacities, but also more profound modifications like the
    redefinition of constraints.
    '''

    def __init__(self, ml):
        '''
        To initialize, the ModelBase object is made an instance attribute.
        '''

        self.ml = ml

        self.fn_def_run, self.csv_df_out = self.ml.get_def_run_name()

    def select_storage_energy_cost(self, dict_ec):

        slct_ec = dict_ec[self.ml.dct_step['swec']]

        vals_ec = ({'HYD_STO': 1e-1, 'NEW_STO': 1e-2} if slct_ec == 'early_phs'
                   else {'HYD_STO': 1e-2, 'NEW_STO': 1e-1})

        df_data = self.ml.m.df_plant_encar.loc[self.ml.m.df_plant_encar.pp_id.isin(self.ml.m.st)]
        df_data = self.ml.m.mps.id_to_name(df_data).set_index(['pp_id', 'ca_id']).pt.replace(vals_ec).rename('ec')

        par = self.ml.m.dict_par['ec']
        par.init_update(df_data.reset_index())

        self.ml.dct_vl['swec_vl'] = str(slct_ec)


    def select_new_storage_capacity(self, dict_st):

        slct_st = dict_st[self.ml.dct_step['swst']]

        list_pt_hyd = self.ml.m.df_def_pp_type.query('pt in ["HYD_STO"]').pt_id.tolist()
        list_pt_new = self.ml.m.df_def_pp_type.query('pt in ["NEW_STO"]').pt_id.tolist()
        ids_hyd_sto = self.ml.m.df_def_plant.query(f'pt_id in {list_pt_hyd}').pp_id.tolist()
        ids_new_sto = self.ml.m.df_def_plant.query(f'pt_id in {list_pt_new}').pp_id.tolist()

        cols = ['pp_id', 'ca_id']
        df_cap = self.ml.io.modwr.dict_comp_obj['cap_pwr_leg'].get_df().set_index(cols).value.rename('cap')
        df_cap = df_cap.loc[ids_hyd_sto].reset_index()
        df_cap = self.ml.m.mps.id_to_name(df_cap).set_index('nd_id').cap
        df_cap = df_cap.reset_index()

        df_dd = self.ml.io.modwr.dict_comp_obj['discharge_duration'].get_df()[cols + ['value']]
        df_dd = df_dd.loc[df_dd.pp_id.isin(ids_hyd_sto + ids_new_sto)]
        df_dd = self.ml.m.mps.id_to_name(df_dd)
        df_dd = df_dd.pivot_table(index='nd_id', columns='fl', values='value')
        df_dd = df_dd.rename(columns=lambda x: f'dd_{x}').reset_index()

        df = pd.merge(df_cap, df_dd)

        df['scale'] = slct_st

        df['cap_new'] = df.cap * df.dd_pumped_hydro * df.scale / df.dd_new_storage

        map_pp_id = self.ml.m.df_def_plant.query(f'pp_id in {ids_new_sto}').set_index('nd_id').pp_id

        df = (df.join(map_pp_id, on='nd_id')
                .assign(ca_id=0, cap_pwr_leg=lambda x: x.cap_new))[['pp_id', 'ca_id', 'cap_pwr_leg']]

        par = self.ml.m.dict_par['cap_pwr_leg']
        par.init_update(df)

        self.ml.dct_vl['swst_vl'] = f'{slct_st*100:.1f}%'

    def select_frozen_params(self, dict_fz):

        slct_fz = dict_fz[self.ml.dct_step['swfz']]

        self.ml.dct_vl['swfz_vl'] = str(slct_fz)

        return slct_fz
    

    def set_future_year(self, dict_fy, slct_fz='default'):

        rng_fy = np.arange(len(dict_fy))
        rng_fyp = np.ones(len(rng_fy)) * {val: key for key, val in dict_fy.items()}[2015]
        dict_previous = dict(zip(rng_fy, rng_fyp))

        slct_fy = self.ml.dct_step['swfy']

        str_fy = '_yr' + str(dict_fy[slct_fy]) if dict_fy[slct_fy] != 2015 else ''
        str_fyp = '_yr' + str(dict_fy[dict_previous[slct_fy]]) if dict_fy[dict_previous[slct_fy]] != 2015 else ''

        @wrapt.decorator
        def log(f, self, args, kwargs):
            str_fy, slct_col, mt_fact = f(*args, **kwargs)

            msg = f"Setting vc_fl to values {str_fy.replace('_', ' ') if str_fy else 'default'}"
            msg += f' from value column "{slct_col}"' if slct_col else ''
            msg += f' using monthly adjustment col "{mt_fact}"' if mt_fact else ''

            logger.info(msg)

        #######################################################################
        @log
        def set_fuel_prices(str_fy=None):
            ''' Select fuel price values for selected year. '''

            if not str_fy:
                str_fy = ''

            slct_col = 'vc_fl' + str_fy

            par = self.ml.m.dict_par['vc_fl']
            df_new = self.ml.m.df_fuel_node_encar[['fl_id', 'nd_id', slct_col]]
            df_new = df_new.rename(columns={slct_col: 'vc_fl'})
            col_mt_fact = 'mt_fact' if not str_fy else 'mt_fact_others'
            
            par.init_update(df_new, col_mt_fact)

            return str_fy, slct_col, col_mt_fact

        #######################################################################
        @log
        def set_cap_pwr_leg(str_fy=None):
            ''' Select power plant capacities for selected year. '''

            slct_col = 'cap_pwr_leg' + str_fy

            par = self.ml.m.dict_par['cap_pwr_leg']
            cols = par.index_cols + [slct_col]
            data_cap = self.ml.m.df_plant_encar.loc[:, cols].copy()
            col_rename = {slct_col: par.value_col}
            data_cap = data_cap.rename(columns=col_rename)

            par.init_update(data_cap)

            return str_fy, slct_col, None

        #######################################################################
        @log
        def set_cap_avlb(str_fy=None):

            if str_fy == None:
                str_fy = ''

            col_mt_fact = 'mt_fact' if not str_fy else 'mt_fact_others'

            msg = ('Setting cap_avlb monthly adjustment to values'
                   + ' from column {}'.format(col_mt_fact))

            logger.info(msg)

            par = self.ml.m.dict_par['cap_avlb']
            mask_pp = self.ml.m.df_plant_encar.pp_id.isin(self.ml.m.pp)
            df_new = self.ml.m.df_plant_encar.loc[mask_pp,
                                                  ['pp_id', 'ca_id', 'cap_avlb']]

            par.init_update(df_new, col_mt_fact)

            return str_fy, None, col_mt_fact

        #######################################################################
        @log
        def set_dmnd(str_fy=None):
            ''' Scale demand profiles for selected year. '''

            if str_fy == None:
                str_fy = ''

            slct_col = 'dmnd_sum' + str_fy
            slct_col_prev = 'dmnd_sum' + str_fyp

            last_dmnd = self.ml.m.df_node_encar.set_index(['nd_id', 'ca_id'])[slct_col_prev]
            next_dmnd = self.ml.m.df_node_encar.set_index(['nd_id', 'ca_id'])[slct_col]

            scaling_factor = (next_dmnd / last_dmnd).rename('scale')


            df = IO.param_to_df(self.ml.m.dmnd)
            df = df.join(scaling_factor, on=scaling_factor.index.names)
            df['value'] = df.value * df.scale

            data_dmnd = df[['sy', 'nd_id', 'ca_id', 'value']]

            par = self.ml.m.dict_par['dmnd']
            par.init_update(data_dmnd)

            return str_fy, slct_col, None

        #######################################################################
        @log
        def set_co2_price(str_fy=None):
            ''' Select CO2 price for selected year. '''

            slct_col = 'price_co2' + str_fy

            df_new = self.ml.m.df_def_node[['nd_id', slct_col]]
            par = self.ml.m.dict_par['price_co2']
            col_mt_fact = 'mt_fact' if not str_fy else 'mt_fact_others'
            par.init_update(df_new, col_mt_fact)

            return str_fy, slct_col, col_mt_fact

        #######################################################################
        @log
        def set_erg_inp(str_fy=None):
            ''' Select exogenous energy production for selected year. '''

            slct_col = 'erg_inp' + str_fy
            par = self.ml.m.dict_par['erg_inp']
            data_erg_inp = (self.ml.m.df_fuel_node_encar[par.index_cols + [slct_col]]
                                   .rename(columns={slct_col: par.value_col}))

            par.init_update(data_erg_inp)

            return str_fy, slct_col, None

        #######################################################################

        @log
        def set_erg_chp(str_fy=None):
            ''' Select exogenous chp energy production for selected year. '''

            slct_col = 'erg_chp' +  str_fy

            par = self.ml.m.dict_par['erg_chp']
            mask_chp = self.ml.m.df_plant_encar.pp_id.isin(self.ml.m.chp)
            data_erg_chp = (
                self.ml.m.df_plant_encar.loc[mask_chp, ['pp_id', 'ca_id', slct_col]]
                         .rename(columns={slct_col: par.value_col}))
            par.init_update(data_erg_chp)

            return str_fy, slct_col, None


        if not slct_fz == 'frz_vc':
            set_fuel_prices(str_fy)
        if not slct_fz == 'frz_cap':
            set_cap_pwr_leg(str_fy)
        if not slct_fz == 'frz_cap':
            set_cap_avlb(str_fy)
        if not slct_fz == 'frz_cap':
            set_dmnd(str_fy)
        if not slct_fz == 'frz_vc':
            set_co2_price(str_fy)
        if not slct_fz == 'frz_cap':
            set_erg_inp(str_fy)
        if not slct_fz == 'frz_cap':
            set_erg_chp(str_fy)

        self.ml.dct_vl['swfy_vl'] = 'yr' + str(dict_fy[slct_fy])

# Run the model

In [ ]:
def run_model(run_id):

    mlm = ModelLoopModifier(ml)

    ml.select_run(run_id)

    logger.info('reset_parameters')
    ml.m.reset_all_parameters()
    ml.m.add_transmission_bounds_rules()
    ml.m.supply.activate()
    ml.m.erg_store_level.activate()
    ml.m.st_erg_capac.activate()
    ml.m.ppst_capac.activate()
    ml.m.unfix_all_vars()

    logger.info('select_frozen_params')
    slct_fz = mlm.select_frozen_params(dict_fz=dict_fz)
    logger.info('set_future_year')
    mlm.set_future_year(dict_fy=dict_fy, slct_fz=slct_fz)
    logger.info('select_new_storage_capacity')
    mlm.select_new_storage_capacity(dict_st=dict_st)
    logger.info('select_storage_preference')
    mlm.select_storage_energy_cost(dict_ec=dict_ec)

    #########################################
    ############### RUN MODEL ###############

    df_vc_om = IO.param_to_df(ml.m.vc_om, ['pp_id', 'ca_id'])
    df_vc_om = df_vc_om.assign(pp=df_vc_om.pp_id.replace(ml.m.mps.dict_pp))
    df_vc_om = df_vc_om.loc[df_vc_om.pp.str.contains('SFH_.._STO')]

    for key, val in df_vc_om.set_index(['pp_id', 'ca_id']).value.to_dict().items():
        ml.m.vc_om[key] = 0.1

    logger.info('fill_peaker_plants')
    ml.m.fill_peaker_plants(demand_factor=5,
                            list_peak=[(ml.m.mps.dict_pp_id['CH_GAS_LIN'], 0)]
                            if not ml.m.setlst['peak'] else []
                            )

    logger.info('_limit_prof_to_cap')
    ml.m._limit_prof_to_cap()

    # ~~~~~~~~~~~~~~~~~~~ VRE capacity fixed --> fix corresponding pwr

    logger.info('fixing VRE pwr')
    dfcap = IO.param_to_df(ml.m.cap_pwr_leg).rename(columns={'value': 'cap'})
    dfcap = dfcap.loc[dfcap.pp_id.isin(ml.m.pr)].set_index('pp_id').cap
    dfprf = IO.param_to_df(ml.m.supprof).rename(columns={'value': 'cf'})
    dfprf = dfprf.join(dfcap, on=dfcap.index.names)
    dfprf['value'] = dfprf.cf * dfprf.cap

    dict_val = dfprf.set_index(['sy', 'pp_id', 'ca_id']).value.to_dict()

    for key in ml.m.supprof:
        # if this has been fixed by swcp already, don't touch it
        if not ml.m.pwr[key].fixed:
            ml.m.pwr[key] = dict_val[key]
            ml.m.pwr[key].fix()

    ml.perform_model_run()

ml.m.solver.options['barrier convergetol'] = 5e-9

from grimsel.auxiliary.multiproc import run_parallel, run_sequential
print('RUNNING')
run_parallel(ml, run_model, 10, groupby=['swfy', 'swec'], adjust_logger_levels=True)
#run_sequential(ml, run_model)